In [3]:
import pandas as pd

file_path = r"C:\Users\BOOK 1\Downloads\Data Stata3.xlsx"
xls = pd.ExcelFile(file_path)

print("=== TÊN CÁC SHEET ===")
print(xls.sheet_names)
print()

=== TÊN CÁC SHEET ===
['Mật độ dân số', 'Tuổi thọ trung bình', 'Tỷ lệ thất nghiệp', 'GDP_người', 'IIP', 'Doanh nghiệp_1000ng', 'Thu nhập du lịch', 'Thu nhập bình quân', 'GINI', 'Tỷ lệ hộ nghèo', 'Tỷ lệ sd nước sạch']



In [4]:
for sheet in xls.sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    n_non_empty = df.dropna(how="all").shape[0]
    print(f"{sheet:30s}  -->  so dong co du lieu: {n_non_empty}")

Mật độ dân số                   -->  so dong co du lieu: 64
Tuổi thọ trung bình             -->  so dong co du lieu: 64
Tỷ lệ thất nghiệp               -->  so dong co du lieu: 64
GDP_người                       -->  so dong co du lieu: 64
IIP                             -->  so dong co du lieu: 64
Doanh nghiệp_1000ng             -->  so dong co du lieu: 64
Thu nhập du lịch                -->  so dong co du lieu: 64
Thu nhập bình quân              -->  so dong co du lieu: 64
GINI                            -->  so dong co du lieu: 64
Tỷ lệ hộ nghèo                  -->  so dong co du lieu: 64
Tỷ lệ sd nước sạch              -->  so dong co du lieu: 64


In [1]:
import pandas as pd

# ==== 0) READ FILE ====
file_path = r"C:\Users\BOOK 1\Downloads\Data Stata3.xlsx"
xls = pd.ExcelFile(file_path)

# ==== 1) Đọc tất cả sheet và đưa về long ====
all_list = []

for sheet in xls.sheet_names:
    # đọc sheet
    df = pd.read_excel(file_path, sheet_name=sheet)

    # đổi tên chỉ tiêu (tên sheet → dùng làm Indicator)
    indicator_name = sheet.strip()

    # melt wide → long
    df_long = df.melt(
        id_vars=df.columns[0],   # cột đầu tiên = Province
        var_name="Year",
        value_name="Value"
    )

    # rename
    df_long = df_long.rename(columns={df.columns[0]: "Province"})
    df_long["Indicator"] = indicator_name

    all_list.append(df_long)

# gộp tất cả indicator vào 1 bảng long
panel_long = pd.concat(all_list, ignore_index=True)

# ==== 2) Chuẩn hoá dữ liệu ====
# strip text
panel_long["Province"] = (
    panel_long["Province"].astype(str).str.strip().str.replace(r"\s+", " ", regex=True)
)
panel_long["Indicator"] = panel_long["Indicator"].astype(str).str.strip()

# convert Year
panel_long["Year"] = panel_long["Year"].astype(int)

# convert Value: bỏ dấu phẩy, cast float
panel_long["Value"] = (
    panel_long["Value"]
    .astype(str).str.replace(",", "").str.strip()
)
panel_long["Value"] = pd.to_numeric(panel_long["Value"], errors="coerce")

# ==== 3) Pivot long -> wide (đúng yêu cầu Q2 = B) ====
panel_wide = panel_long.pivot_table(
    index=["Province", "Year"],
    columns="Indicator",
    values="Value",
    aggfunc="mean"  # nếu có duplicated (rất hiếm)
).reset_index()

panel_wide.columns.name = None  # bỏ tên multiindex

# ==== 4) Xem kết quả ====
print(panel_wide.head(15))
print("\nShape wide:", panel_wide.shape)
print("\nSố indicator:", len(panel_wide.columns)-2)

             Province  Year  Doanh nghiệp_1000ng  GDP_người  GINI     IIP  \
0            An Giang  2019                  2.0       44.5  0.34  109.86   
1            An Giang  2020                  3.0       46.6  0.32  103.28   
2            An Giang  2021                  3.0       47.7  0.36  103.10   
3            An Giang  2022                  3.0       52.7  0.35  111.09   
4            An Giang  2023                  3.0       58.8  0.36  108.40   
5   Bà Rịa - Vũng Tàu  2019                  9.0      300.4  0.40  102.07   
6   Bà Rịa - Vũng Tàu  2020                  9.0      251.2  0.39   93.31   
7   Bà Rịa - Vũng Tàu  2021                 10.0      297.6  0.39   95.45   
8   Bà Rịa - Vũng Tàu  2022                 10.0      372.1  0.37  105.65   
9   Bà Rịa - Vũng Tàu  2023                 10.0      345.4  0.37   99.43   
10         Bình Dương  2019                 13.0      147.4  0.28  109.02   
11         Bình Dương  2020                 13.0      150.5  0.28  106.10   

In [5]:
# FULL PIPELINE: clean -> flag missing -> export .dta -> EDA -> prepare panel vars
import pandas as pd
import unidecode
import numpy as np
import os

# ----- USER PATHS: chỉnh nếu cần -----
file_path = r"C:\Users\BOOK 1\Downloads\Data Stata3.xlsx"
out_stata = r"C:\Users\BOOK 1\Downloads\panel_wide.dta"   # file .dta xuất ra

# ----- 1) Read & melt all sheets to LONG -----
xls = pd.ExcelFile(file_path)
all_list = []
for sheet in xls.sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    df = df.dropna(how="all")   # drop hoàn toàn trống
    # melt wide -> long, cột đầu là Province (header đã có)
    df_long = df.melt(id_vars=df.columns[0], var_name="Year", value_name="Value")
    df_long = df_long.rename(columns={df.columns[0]: "Province"})
    df_long["Indicator"] = sheet.strip()
    all_list.append(df_long)

panel_long = pd.concat(all_list, ignore_index=True)

# ----- 2) Clean text / types -----
# Province: strip, collapse spaces, remove accents (P2)
panel_long["Province"] = (
    panel_long["Province"].astype(str)
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .apply(lambda x: unidecode.unidecode(x))
)

# Indicator: normalize to ascii_snake_case variable names for wide columns
panel_long["Indicator_clean"] = (
    panel_long["Indicator"]
    .astype(str)
    .str.strip()
    .apply(lambda s: unidecode.unidecode(s).lower().replace(" ", "_").replace("%","pct"))
)

# Year -> int
panel_long["Year"] = panel_long["Year"].astype(int)

# Value: remove grouping thousands (commas/space/dots as thousand) and coerce numeric where possible
panel_long["Value_raw"] = panel_long["Value"].astype(str).str.strip()

# Try common replacements: remove spaces, replace comma decimal if needed
# Approach: remove thousand separators (commas or spaces) but keep decimal dot
panel_long["Value_num"] = (
    panel_long["Value_raw"]
    .str.replace(r"\s+", "", regex=True)      # remove spaces
    .str.replace(r"(?<=\d),(?=\d{3}\b)", "", regex=True)   # remove thousand-comma (e.g. 1,234 -> 1234)
    .str.replace(",", ".", regex=False)       # replace remaining commas (decimal comma) -> dot
)

panel_long["Value_num"] = pd.to_numeric(panel_long["Value_num"], errors="coerce")

# Replace numeric column and drop helper raw
panel_long["Value"] = panel_long["Value_num"]
panel_long = panel_long.drop(columns=["Value_raw","Value_num"])

# ----- 3) Flag missing (M4): tạo cột flag per-row & summary -----
panel_long["is_missing"] = panel_long["Value"].isna().astype(int)

# summary missing by indicator and overall
missing_by_ind = panel_long.groupby("Indicator_clean")["is_missing"].sum().sort_values(ascending=False)
total_missing = panel_long["is_missing"].sum()
total_obs = len(panel_long)

print("=== Missing summary (top indicators) ===")
print(missing_by_ind.head(20))
print(f"\nTotal obs: {total_obs}, Total missing cells: {total_missing}\n")

# ----- 4) Pivot to WIDE as requested (D1) -----
panel_wide = panel_long.pivot_table(
    index=["Province","Year"],
    columns="Indicator_clean",
    values="Value",
    aggfunc="mean"   # safe if duplicates
).reset_index()

panel_wide.columns.name = None

# ----- 5) After pivot: create missing flags summary in wide form -----
# count missing indicators by row
indicators = [c for c in panel_wide.columns if c not in ["Province","Year"]]
panel_wide["n_missing"] = panel_wide[indicators].isna().sum(axis=1)
panel_wide["pct_missing"] = panel_wide["n_missing"] / len(indicators)

print("=== panel_wide sample ===")
print(panel_wide.head(8))
print("\nShape wide:", panel_wide.shape)
print("Number of indicators:", len(indicators))

# ----- 6) Export to Stata (.dta) - ensure varnames <= 32 chars and valid -----
# sanitize column names (Stata rules): letters, numbers, _ ; max 32 chars
def stata_safe_name(name):
    s = str(name)
    s = s.replace(" ", "_")
    s = "".join(ch for ch in s if (ch.isalnum() or ch == "_"))
    return s[:31]

# build mapping and rename
rename_map = {c: stata_safe_name(c) for c in panel_wide.columns}
panel_wide_stata = panel_wide.rename(columns=rename_map)

# to_stata requires pandas >= 1.0; include version-safe export
panel_wide_stata.to_stata(out_stata, write_index=False)
print(f"\nExported wide panel to Stata file: {out_stata}")

# ----- 7) EDA mô tả nhanh -----
print("\n=== EDA SUMMARY ===")
# basic describe numeric indicators
desc = panel_wide[indicators].describe().transpose()
print(desc[["count","mean","std","min","25%","50%","75%","max"]])

# top correlations (pairwise) - only numeric (drop indicators with all NaN)
corr_df = panel_wide[indicators].dropna(axis=1, how="all").corr()
print("\nTop correlations (abs) sample (showing strongest pairs):")
# show top 10 absolute correlations
corr_unstack = corr_df.abs().unstack().dropna()
corr_unstack = corr_unstack[corr_unstack < 1].sort_values(ascending=False).drop_duplicates()
print(corr_unstack.head(10))

# time trend: mean indicator by Year for a few indicators (if exist)
sample_inds = list(corr_df.columns[:5])  # pick first 5 indicators that exist
if sample_inds:
    mean_by_year = panel_wide.groupby("Year")[sample_inds].mean()
    print("\nMean by Year for sample indicators:")
    print(mean_by_year)

# missing pattern by province (top provinces with missing)
miss_by_prov = panel_wide.groupby("Province")["n_missing"].sum().sort_values(ascending=False)
print("\nTop provinces by total missing cells:")
print(miss_by_prov.head(10))

# ----- 8) Prepare dataset for panel regression -----
# We'll prepare a regression-ready long dataset (panel_long_reg) with:
# - province_id (numerical)
# - year (int)
# - optionally log_gdp and growth if gdp_nguoi exists
panel_long_reg = panel_wide.copy()

# make a province id
prov_codes = {p: i+1 for i,p in enumerate(sorted(panel_long_reg["Province"].unique()))}
panel_long_reg["province_id"] = panel_long_reg["Province"].map(prov_codes)

# example: if gdp_nguoi exists, create log_gdp and growth rate
gdp_var = None
for cand in ["gdp_nguoi","gdp_nguoi_" , "gdp"]:
    if cand in panel_long_reg.columns:
        gdp_var = cand
        break
# better: find closest match
if gdp_var is None:
    matches = [c for c in panel_long_reg.columns if "gdp" in c]
    if matches:
        gdp_var = matches[0]

if gdp_var:
    panel_long_reg = panel_long_reg.sort_values(["Province","Year"])
    panel_long_reg["log_"+gdp_var] = np.log(panel_long_reg[gdp_var])
    panel_long_reg["growth_"+gdp_var] = panel_long_reg.groupby("Province")[gdp_var].pct_change()
    panel_long_reg["lag1_"+gdp_var] = panel_long_reg.groupby("Province")[gdp_var].shift(1)
    print(f"\nPrepared regression vars based on '{gdp_var}': created log_, growth_ and lag1_ columns.")
else:
    print("\nNo GDP-like variable found automatically; skip creating log/growth/lag variables.")

# Save a regression-ready long dataset to disk as CSV (you can load in Stata too)
out_csv = r"C:\Users\BOOK 1\Downloads\panel_wide_regression_ready.csv"
panel_long_reg.to_csv(out_csv, index=False)
print(f"Saved regression-ready CSV: {out_csv}")

# final message
print("\n=== Pipeline finished. Next suggestions: ===")
print(" - Open the exported .dta in Stata to inspect variable names and missing pattern.")
print(" - If you want, I can run specific EDA plots (time series for provinces, histograms, correlation heatmap).")
print(" - If you want, I can create example panel regression code (Stata xtreg or Python linearmodels) using selected dependent & independent variables.")

=== Missing summary (top indicators) ===
Indicator_clean
iip                    5
gdp_nguoi              0
doanh_nghiep_1000ng    0
gini                   0
mat_do_dan_so          0
thu_nhap_binh_quan     0
thu_nhap_du_lich       0
tuoi_tho_trung_binh    0
ty_le_ho_ngheo         0
ty_le_sd_nuoc_sach     0
ty_le_that_nghiep      0
Name: is_missing, dtype: int64

Total obs: 3520, Total missing cells: 5

=== panel_wide sample ===
            Province  Year  doanh_nghiep_1000ng  gdp_nguoi  gini     iip  \
0           An Giang  2019                  2.0       44.5  0.34  109.86   
1           An Giang  2020                  3.0       46.6  0.32  103.28   
2           An Giang  2021                  3.0       47.7  0.36  103.10   
3           An Giang  2022                  3.0       52.7  0.35  111.09   
4           An Giang  2023                  3.0       58.8  0.36  108.40   
5  Ba Ria - Vung Tau  2019                  9.0      300.4  0.40  102.07   
6  Ba Ria - Vung Tau  2020           

In [6]:
import pandas as pd
file = r"C:\Users\BOOK 1\Downloads\panel_wide.dta"  # hoặc Data Stata3.xlsx sau clean
df = pd.read_stata(file)
print(df.columns.tolist())

['Province', 'Year', 'doanh_nghiep_1000ng', 'gdp_nguoi', 'gini', 'iip', 'mat_do_dan_so', 'thu_nhap_binh_quan', 'thu_nhap_du_lich', 'tuoi_tho_trung_binh', 'ty_le_ho_ngheo', 'ty_le_sd_nuoc_sach', 'ty_le_that_nghiep', 'n_missing', 'pct_missing']
